In [1]:
import pandas as pd
import requests
import numpy as np
import json
import asyncio
import dotenv
import boto3
import glob
import os
import logging
from dotenv import load_dotenv
from sqlalchemy import create_engine, text







In [15]:
# chargement du fichier secret.env pour connecter à AWS S3
load_dotenv(".secrets")

AWS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET = os.getenv("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = os.getenv("S3_BUCKET")


s3 = boto3.client("s3", 
                  aws_access_key_id=AWS_KEY, 
                  aws_secret_access_key=AWS_SECRET)



In [ ]:
# On charge les données depuis le S3
s3.download_file(BUCKET_NAME, "dataset/dataset_training_maj.csv", "dataset_maj.csv")
df_transaction = pd.read_csv("dataset_maj.csv")


/tmp/ipykernel_26436/1544565694.py:3: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_transaction = pd.read_csv("dataset_maj.csv")


In [17]:
# Connexion à la base de données PostgreSQL
load_dotenv("secret.env")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASS")
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")

# Créer l'engine SQLAlchemy
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}")

In [ ]:
# création de la table sur neon
create_table_fraud = """
CREATE TABLE IF NOT EXISTS fraud_transaction(
  id SERIAL PRIMARY KEY,
  cc_num BIGINT,
  merchant TEXT,
  category TEXT,
  amt DOUBLE PRECISION,
  first TEXT,
  last TEXT,
  gender TEXT,
  street TEXT,
  city TEXT,
  state TEXT,
  zip INTEGER,
  lat DOUBLE PRECISION,
  long DOUBLE PRECISION,
  city_pop INTEGER,
  job TEXT,
  merch_lat DOUBLE PRECISION,
  merch_long DOUBLE PRECISION,
  is_fraud INTEGER,
  year INTEGER,
  month INTEGER,
  day INTEGER,
  hour INTEGER,
  minute INTEGER,
  second INTEGER,
  dob_year INTEGER,
  dob_month INTEGER,
  dob_day INTEGER,
  timestamp TIMESTAMPTZ,
  is_fraud_pred INTEGER,
  inserted_at TIMESTAMPTZ DEFAULT NOW()
);
"""
with engine.connect() as connection:
    connection.execute(text(create_table_fraud))
    connection.commit()

In [ ]:
# chargement des données dans la table fraud_transaction
df_transaction.to_sql('fraud_transaction', engine, if_exists='append', index=False)

In [ ]:
# lecture de données pour tester la connexion et le chargement
df_yesterday = pd.read_sql("SELECT is_fraud_pred, day, year FROM fraud_transaction WHERE year = 2025 AND day = 17 LIMIT 10", con=engine)

In [24]:
print(df_yesterday)

   is_fraud_pred  day  year
0              0   17  2025
1              0   17  2025
2              0   17  2025
3              0   17  2025
4              0   17  2025
5              0   17  2025
6              0   17  2025
7              0   17  2025
8              0   17  2025
9              0   17  2025
